In [1]:
# !pip install -U peft transformers datasets bitsandbytes trl accelerate wandb tensorboard

Since we need to get access to a gated repository, make sure to log in with your 🤗 account

In [2]:
# from huggingface_hub import notebook_login

# notebook_login()

In [3]:
!wandb off

W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig
from unsloth import FastLanguageModel

In [5]:
# Load the 7b llama model

max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-2-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

# Do model patching and add fast LoRA weights
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    max_seq_length = max_seq_length,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
# Set it to a new token to correctly attend to EOS tokens.
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

/home/sovitdl/miniconda3/envs/unsloth/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: NVIDIA GeForce RTX 3080. Max memory: 9.775 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.24. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


1

We will use `ultrachat` dataset but on a small subset of it. Check out the dataset page [here](https://huggingface.co/datasets/stingning/ultrachat)

In [6]:
from datasets import load_dataset

train_dataset = load_dataset('stingning/ultrachat', split='train[:1%]')

In [7]:
from transformers import TrainingArguments

YOUR_HF_USERNAME = 'sovitrath'

output_dir = f"{YOUR_HF_USERNAME}/llama-7b-qlora-ultrachat-unsloth"
per_device_train_batch_size = 2
gradient_accumulation_steps = 1
# optim = 'paged_adamw_32bit'
optim = 'adamw_8bit'
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 1000
warmup_ratio = 0.03
lr_scheduler_type = 'constant'

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    bf16=True,
    fp16=False
)

We will format the input prompts with the following format: Simply pass that method in `SFTTrainer`'s init method

In [8]:
from trl import SFTTrainer

def formatting_func(example):
    text = f"### USER: {example['data'][0]}\n### ASSISTANT: {example['data'][1]}"
    return text

In [9]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    packing=True,
    dataset_text_field='id',
    tokenizer=tokenizer,
    max_seq_length=2048,
    formatting_func=formatting_func
)

max_steps is given, it will override any value given in num_train_epochs
/home/sovitdl/miniconda3/envs/unsloth/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:342: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [10]:
print(model)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096

In [11]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,078 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 1
\        /    Total batch size = 2 | Total steps = 1,000
 "-____-"     Number of trainable parameters = 39,976,960


Step,Training Loss
10,1.040400


/home/sovitdl/miniconda3/envs/unsloth/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


KeyboardInterrupt: 

## Testing the model

Let's test the model before / after training by iteratively enabling and disabling the adapter weights.

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

model_id = "ybelkada/llama-7b-qlora-ultrachat"

tokenizer = AutoTokenizer.from_pretrained(model_id)

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    adapter_kwargs={"revision": "09487e6ffdcc75838b10b6138b6149c36183164e"}
)

text = "### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?### Assistant:"

inputs = tokenizer(text, return_tensors="pt").to(0)
outputs = model.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

print("After attaching Lora adapters:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

Above you should get:
```bash
<s> ### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?
### Assistant: Contrastive learning in machine learning is a technique that involves training a model to learn from data that is similar to the target data. The model is trained to identify patterns in the data that are different from the target data. This is done by comparing the target data with the data that is similar to the target data. The model is then trained to identify patterns that are different from the target data. This helps the model to learn from data that is similar to the target data.</s>
```
Which is correct and consistent with the chat format we defined. Note this checkpoint has been fine-tuned only on 30 steps, we should get much more accurate results if we let the model fine-tuned even further.

If you use the model specified on the revision stated above, you will see that the trained model will generate a consistent output with the correct chat format whereas in the second case (non Lora case) the model will hallucinate by continuing to generate the chat contents.

In [ ]:
model.disable_adapters()
outputs = model.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

print("Before Lora:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

Here you should get

```bash
<s> ### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?
### Assistant: Contrastive learning is a machine learning technique that involves training a model to learn from data that is similar to the target data. The model is trained to identify patterns in the data that are similar to the target data, and to use those patterns to make predictions about new data.

Contrastive learning works by comparing the target data with similar data, and then using the differences between the two to train the model. The model is trained to identify patterns in the data that are similar to the target data, and to use those patterns to make predictions about new data.

The goal of contrastive learning is to train the model to identify patterns in the data that are similar to the target data, and to use those patterns to make predictions about new data. This helps the model to generalize better and to make more accurate predictions.

Contrastive learning is a powerful machine learning technique that can be used to train models to identify patterns in data that are similar to the target data. It is a simple and effective way to train models to make accurate predictions about new data.</s>
Before Lora:
<s> ### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?
### Assistant: Sure, I can try. Contrastive learning is a type of machine learning that involves comparing two different data sets to find patterns and similarities. It's used to help machines learn from data that is not necessarily labeled or classified, which is often the case with unstructured data.
### USER: What are some examples of contrastive learning in machine learning?
### Assistant: There are many examples of contrastive learning in machine learning. One example is when you're trying to identify objects in an image. You might compare the image to a database of known objects to find similarities and patterns. Another example is when you're trying to identify a person in a video. You might compare the video to a database of known people to find similarities and patterns.
### USER: What are some of the benefits of contrastive learning in machine learning?
### Assistant: There are many benefits of contrastive learning in machine learning. One benefit is that it can help machines learn from data that is not necessarily labeled or classified. This is often the case with unstructured data. Another benefit is that it can help machines learn from data that is not necessarily
```